![](https://img.kaikeba.com/web/kkb_index/img_index_logo.png)

# Lesson-01 Assignment

   各位同学大家好，欢迎各位开始学习我们的人工智能课程。这门课程假设大家不具备机器学习和人工智能的知识，但是希望大家具备初级的Python编程能力。根据往期同学的实际反馈，我们课程的完结之后 能力能够超过80%的计算机人工智能/深度学习方向的硕士生的能力。

## 本次作业的内容

### 1. Recode all examples;

### 2. Please answer some questions about our course. We do appreciate your help.

   2.1  What do you want to get in this course? 

Answer:知识、经验、技术

2.2 What problems do you want to solve? 

Answer:就业，学习

 2.3 What advantages do you have to accomplish your goal?

Answer:熟悉python，学习能力强

 2.4 What disadvantages you need to overcome to accomplish your goal?

Answer:对专业知识了解不深入

2.5 How will you plan to study in this course? 

Answer:按计划完成学习目标，做好笔记，课外扩充相关知识

### 3. 如何提交

答疑平台提交，具体方式，见作业提交指南

### 4. 作业截止时间

作业能帮助你回顾课堂内容，你又可以通过作业进行代码实操。咱们可要认真、及时的完成作业哦！自布置作业起两周内提交，助教及时批改作业哦～逾期提交不批改。（特殊情况，请找班主任请假。）

### 5. 完成以下问答和编程练习

5.1  Please combine **image crop, color shift, rotation and perspective transform** together to complete a data augmentation script.
   Your code need to be completed in Python/C++ in .py or .cpp file with comments and readme file to indicate how to use.

In [204]:
import cv2
import math
import random
import numpy as np

def image_crop(image, x_start, x_end, y_start, y_end): 
    """
    @param image: 操作图片
    @param x_start: x起始点
    @param x_end:   x结束点
    @param y_start: y起始点
    @param y_off:   y结束点
    超过边界自动转到图像边界
    """
    y_lim, x_lim, _ = image.shape
    x_start = 0 if x_start < 0 else x_start
    y_start = 0 if y_start < 0 else y_start
    if x_start > x_end or y_start > y_end:
        raise Exception("illegal param, please check your boundary")
    x_end = x_lim if x_end > x_lim else x_end
    y_end = y_lim if y_end > y_lim else y_end
    return image[y_start:y_end, x_start:x_end,:]

def image_move(image, right, down): 
    """
    @param image: 原始图片
    @param right: 向右移动距离
    @param down:  向下移动距离
    负数表示向反方向移动
    """
    move_matrix = np.array([[1, 0, right], [0, 1, down]]).astype('float32')
    return cv2.warpAffine(image, move_matrix, image.shape[-2::-1])

def my_rotation(image, angle, clockwise=True):
    """
    自定义旋转函数
    @param image: 原始图片
    @param angle: 旋转角度
    @param clockwose: 旋转方向
    """
    if not clockwise:
        angle = -angle
    y, x, _ = image.shape 
    y, x = y / 2, x / 2
    r = np.linalg.norm([x/2.0, y/2.0])
    _angle = math.pi * angle / 180.0
    _cos = math.cos(_angle)
    _sin = math.sin(_angle)
    right = x * (1 - _cos) + y * _sin
    down =  x * (1 - _sin) - y * _cos
    print(right, down)
    rotation_matrix = np.array([[_cos, -_sin, right], [_sin, _cos, down]])
    return cv2.warpAffine(image, rotation_matrix, image.shape[-2::-1])

def system_rotation(image, angle, clockwise=False):
    """
    系统调用组装旋转函数
    @param image: 原始图片
    @param angle: 旋转角度
    @param clockwose: 旋转方向
    """
    if clockwise:
        angle = -angle
    y, x, _ = image.shape
    center = (x / 2, y / 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    return cv2.warpAffine(image, rotation_matrix, (x, y))

color_dict = {
    'red':{
        'h':[(0,10),(156,180)]
    },
    'orange':{
        'h':[(11,25)]
    },
    'yellow':{
        'h':[(26,34)]
    },
    'green':{
        'h':[(35,77)]
    },
    'cyan':{
        'h':[(35,77)]
    },
    'blue':{
        'h':[(100,124)]
    },
    'purple':{
        'h':[(125,155)]
    }
}
support = color_dict.keys()
base_h = [(0, 180)]
# 剔除了黑白灰，暂时用不到
base_s = (43, 255)
base_v = (46, 255)
def color_shift(image, _from=None, _to=None):
    """
    @param image:HSV格式图像
    @param _from: 旧颜色
    @param _to: 新颜色
    """
    global support
    global special
    
    h,s,v = cv2.split(image)
    if (_from not in support) or (_to not in support):
        raise Exception('unsupport color, support list : {0}'.format(support))
    if _from == _to:
        return cv2.merge((h,s,v))
    from_info = color_dict[_from]
    to_info = color_dict[_to]
    from_h = from_info.setdefault('h', base_h)[-1]
    to_h   = to_info.setdefault('h', base_h)[-1]
    for _from_h in from_h:
        h = boundary_mapping(h, from_h, to_h)
    return cv2.merge((h,s,v))

def boundary_mapping(value, old_boundary, new_boundary,min_offset=10, max_offset=10):
    old_min, old_max = old_boundary
    old_min, old_max = old_min-min_offset, old_max +max_offset
    new_min, new_max = new_boundary
    old_scale = old_max - old_min
    new_scale = new_max - new_min
    for y in range(len(value)):
        for x in range(len(value[y])):
            if old_min <= value[y][x] <= old_max:
                value[y][x] = (value[y][x] - old_min) / old_scale * new_scale + new_min

    return value.astype('uint8')



def perspective_transform(image,*args): 
    if not len(args) == 16:
        raise Exception("illegal args, need 16 int")
    point_paire = np.array(args).reshape(2,4,2).astype('float32')
    print(point_paire)
    mirror_matrix = cv2.getPerspectiveTransform(point_paire[0], point_paire[1])
    return cv2.warpPerspective(image, mirror_matrix, image.shape[-2::-1])

各位同学，你已经把课上关于图像增广实现了！CV的领域很广，咱们需要思考兴趣点在哪～

5.2  Do think about your own interests very carefully and choose your topic within 3 weeks.

Answer:图像的各种处理和应用以及各种神经网络、机器学习

这次的作业就到这里了！祝大家学习进步！

![image alt <](http://5b0988e595225.cdn.sohucs.com/images/20190420/1d1070881fd540db817b2a3bdd967f37.gif)